<a href="https://colab.research.google.com/github/wolfzxcv/ml-examples/blob/master/TSP_using_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
from random import randint, choice
from math import exp

# A very high value to represent no direct path between cities
INT_MAX = 2147483647

# Number of cities in TSP
V = 5

# Names of the 5 cities
#GENES = "ABCDE"

# Starting city index (0, representing city 'A')
START = 2

# Initial population size (potential solutions) for the algorithm
POP_SIZE = 10

In [117]:
# individual solution (a possible path)
class Individual:
  def __init__(self, gnome, fitness):
    # A string representing the path (e.g., "012340")
    self.gnome = gnome
    # The total distance of the path
    self.fitness = fitness


# Generate a random number between start and end-1
def rand_num(start, end):
    return randint(start, end - 1)  # Adjust for Python's zero-based indexing


# Check if a character ch is already in string s
def repeat(s, ch):
  return ch in s


# Mutates a gnome by swapping two random cities in the path
def mutated_gene(gnome):
    gnome = list(gnome)
    while True:
        r = rand_num(1, V)
        r1 = rand_num(1, V)
        if r1 != r:
            temp = gnome[r]
            gnome[r] = gnome[r1]
            gnome[r1] = temp
            break
    return ''.join(gnome)


# Generates a valid initial path (gnome) starting and ending at the first city
def create_gnome():
    gnome = str(choice(range(V)))
    while len(gnome) < V:
      temp = rand_num(0, V)
      if str(temp) not in gnome:
        gnome += str(temp)
    return gnome


# Calculates the fitness of a gnome by summing the distances between continuous cities in the path
def calculate_fitness(gnome):
    mp = [
        [0, 2, INT_MAX, 12, 5],
        [2, 0, 4, 8, INT_MAX],
        [INT_MAX, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, INT_MAX, 3, 10, 0],
    ]
    fitness = 0
    for i in range(V - 1):
      fitness += mp[int(gnome[i])][int(gnome[i + 1])]
    return fitness


# Reduces the "temperature" as the algorithm progresses, result in less likely to accept worse solutions over time.
def cooldown(temp):
    return int(temp * 0.9)

In [118]:
# Main function for TSP problem.
def tsp_util():

    population = []

    temperature = 10000

    # Generation Number
    gen = 1

    # Number of Gene Iterations
    gen_thres = 10


    print("Initial population: \nGNOME FITNESS VALUE")
    # Populating the GNOME pool.
    for i in range(POP_SIZE):
        gnome = create_gnome()
        fitness = calculate_fitness(gnome)
        print(gnome, fitness)
        population.append(Individual(gnome, fitness))

    # Iteration to perform
    # population crossing and gene mutation.
    while temperature > 1000 and gen <= gen_thres:
      population.sort(key=lambda x: x.fitness)
      new_population = []
      print("\nCurrent temperature: ", temperature)
      for p1 in population:
        while True:
          new_g = mutated_gene(p1.gnome)
          new_fitness = calculate_fitness(new_g)
          new_gnome = Individual(new_g, new_fitness)
          if new_gnome.fitness <= p1.fitness:
            new_population.append(new_gnome)
            break
          else:
            delta = new_fitness - p1.fitness
            prob = exp(-delta / temperature)
            if prob > 0.5:
              new_population.append(new_gnome)
              break
      temperature = cooldown(temperature)
      population = new_population

      print(f"Generation {gen}")
      print("GNOME FITNESS VALUE")

      for ind in population:
        print(f"{ind.gnome} {ind.fitness}")
      # Print the smallest fitness value (shortest path) after the last iteration
      if gen == gen_thres:
        smallest_fitness = min(ind.fitness for ind in population)
        print(f"\nShortest path (smallest fitness value in the last iteration): {smallest_fitness}")

      gen += 1


if __name__ == "__main__":
    tsp_util()

Initial population: 
GNOME FITNESS VALUE
01234 19
21340 27
30142 2147483664
34021 2147483666
32410 2147483655
31042 18
23401 20
41302 4294967314
02134 2147483669
13420 2147483668

Current temperature:  10000
Generation 1
GNOME FITNESS VALUE
34012 21
01324 16
23104 18
24310 23
32401 13
31042 18
30421 24
10423 13
04132 2147483663
41203 4294967310

Current temperature:  9000
Generation 2
GNOME FITNESS VALUE
32104 14
10432 20
01234 19
23401 20
34012 21
34210 19
21340 27
30124 21
04312 27
41302 4294967314

Current temperature:  8100
Generation 3
GNOME FITNESS VALUE
32401 13
01243 19
31240 20
12430 29
24301 27
31042 18
30421 24
24310 23
01342 23
40312 29

Current temperature:  7290
Generation 4
GNOME FITNESS VALUE
32104 14
34012 21
01342 23
31042 18
24013 18
04312 27
30124 21
23401 20
10432 20
43012 28

Current temperature:  6561
Generation 5
GNOME FITNESS VALUE
30124 21
31240 20
24310 23
23104 18
10423 13
31042 18
30421 24
01243 19
04321 22
43210 19

Current temperature:  5904
Generation 6
